In [68]:
import pandas as pd

df = pd.read_pickle('../data/final_data.pkl')

In [69]:
# # ElasticNet PREDICTION TOUS les Types => Pas très bon

# import numpy as np
# from sklearn.linear_model import ElasticNet
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import r2_score


# df_test = df.copy()

# X = df_test.drop(columns=['Valeur_fonciere','Commune'])
# print(X.columns)
# y = df_test['Valeur_fonciere']

# # Divisez les données en ensembles d'entraînement et de test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Définissez les paramètres pour la recherche sur la grille
# param_grid = {
#     'alpha': [0.1, 1.0, 10.0],  # Valeurs à tester pour le paramètre alpha
#     'l1_ratio': [0.1, 0.5, 0.9]  # Valeurs à tester pour le paramètre l1_ratio
# }

# # Créez un modèle Elastic Net
# elastic_net = ElasticNet()

# # Créez un objet GridSearchCV pour la recherche sur la grille
# grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# # Exécutez la recherche sur la grille sur les données d'entraînement
# grid_search.fit(X_train, y_train)

# # Obtenez les meilleurs paramètres et le meilleur modèle
# meilleurs_parametres = grid_search.best_params_
# meilleur_modele = grid_search.best_estimator_

# # Utilisez le meilleur modèle pour faire des prédictions sur les données de test
# predictions = meilleur_modele.predict(X_test)

# # Calculez l'erreur quadratique moyenne des prédictions de Ridge
# mse = mean_squared_error(y_test, predictions,squared=False)

# print("Meilleurs paramètres pour Elastic Net: ", meilleurs_parametres)
# # Affichez l'erreur quadratique moyenne
# print("Erreur Quadratique Moyenne (MSE) : ", mse)

# # Créez un DataFrame avec les valeurs prédites et les valeurs réelles
# resultats = pd.DataFrame(
#     {'Valeur_reelle': y_test, 'Valeur_predite': predictions})

# # Affichez les valeurs prédites par rapport aux valeurs réelles
# print(resultats)
# # Calculez le R carré
# r2 = r2_score(y_test, predictions)
# # Affichez le R carré
# print("R carré :", r2)

In [70]:
import pandas as pd
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import train_test_split

valeurs_code_type_local = [1, 2, 3, 4]
alphas = [10.0, 0.1, 1.0]  # Toutes les valeurs d'alpha que vous voulez tester
l1_ratios = [0.1, 0.5, 0.9]  # Toutes les valeurs de l1_ratio que vous voulez tester

# Dictionnaire pour stocker les performances par modèle et par valeur de 'Code_type_local', 'alpha' et 'l1_ratio'
performances = {}


# Dictionnaire pour stocker les meilleurs modèles par Code_type_local
meilleurs_modeles = {}

# Boucle sur les valeurs d'alpha et de l1_ratio pour Elastic Net
for alpha in alphas:
    for l1_ratio in l1_ratios:
        for code_type_local in valeurs_code_type_local:
            # Filtrer le DataFrame pour la valeur spécifique de 'Code_type_local'
            sous_ensemble_df = df[df['Code_type_local'] == code_type_local]

            # Divisez les données en caractéristiques (X) et la variable cible (y)
            X = sous_ensemble_df.drop(columns=['Code_type_local', 'Valeur_fonciere','Commune'])  # Remplacez 'variable_cible' par le nom réel de votre variable cible
            y = sous_ensemble_df['Valeur_fonciere']

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Créez le modèle Elastic Net avec les alphas et l1_ratio spécifiques
            elastic_net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
            elastic_net.fit(X_train, y_train)
            elastic_net_predictions = elastic_net.predict(X_test)
            elastic_net_rmse = sqrt(mean_squared_error(
                y_test, elastic_net_predictions))

        
           # Vérifiez si c'est le meilleur modèle pour cette combinaison de paramètres et de Code_type_local
            if code_type_local not in meilleurs_modeles or elastic_net_rmse < meilleurs_modeles[code_type_local]['RMSE']:
                meilleurs_modeles[code_type_local] = {
                    'Modèle': elastic_net,
                    'RMSE': elastic_net_rmse,
                    'Alpha': alpha,
                    'L1_ratio': l1_ratio
                }
   
            modele_key = f'ElasticNet alpha={alpha} l1_ratio={l1_ratio}'
            if modele_key not in performances:
                performances[modele_key] = {}
            performances[modele_key][f'Code_type_local {code_type_local}'] = elastic_net_rmse



# Boucle sur les valeurs d'alpha pour Lasso et Ridge
for alpha in alphas:
    for code_type_local in valeurs_code_type_local:
        # Filtrer le DataFrame pour la valeur spécifique de 'Code_type_local'
        sous_ensemble_df = df[df['Code_type_local'] == code_type_local]

        # Divisez les données en caractéristiques (X) et la variable cible (y)
        X = sous_ensemble_df.drop(columns=['Code_type_local', 'Valeur_fonciere','Commune'])  # Remplacez 'variable_cible' par le nom réel de votre variable cible
        y = sous_ensemble_df['Valeur_fonciere']

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42)
        
        # Créez le modèle Lasso avec l'alpha spécifique
        lasso = Lasso(alpha=alpha)
        lasso.fit(X_train, y_train)
        lasso_predictions = lasso.predict(X_test)
        lasso_rmse = sqrt(mean_squared_error(y_test, lasso_predictions))


        # Vérifiez si c'est le meilleur modèle pour cette combinaison de paramètres et de Code_type_local
        if code_type_local not in meilleurs_modeles or lasso_rmse < meilleurs_modeles[code_type_local]['RMSE']:
            meilleurs_modeles[code_type_local] = {
                'Modèle': lasso,
                'RMSE': lasso_rmse,
                'Alpha': alpha
            }

         # Divisez les données en caractéristiques (X) et la variable cible (y)
        # Remplacez 'variable_cible' par le nom réel de votre variable cible
        X = sous_ensemble_df.drop(
            columns=['Code_type_local', 'Valeur_fonciere', 'Commune'])
        y = sous_ensemble_df['Valeur_fonciere']

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42)
        
        # Créez le modèle Ridge avec l'alpha spécifique
        ridge = Ridge(alpha=alpha)
        ridge.fit(X_train, y_train)
        ridge_predictions = ridge.predict(X_test)
        ridge_rmse = sqrt(mean_squared_error(y_test, ridge_predictions))

       

        # Vérifiez si c'est le meilleur modèle pour cette combinaison de paramètres et de Code_type_local
        if code_type_local not in meilleurs_modeles or ridge_rmse < meilleurs_modeles[code_type_local]['RMSE']:
            meilleurs_modeles[code_type_local] = {
                'Modèle': ridge,
                'RMSE': ridge_rmse,
                'Alpha': alpha
            }

         # Enregistrez les performances dans le dictionnaire
        modele_key = f'Lasso alpha={alpha}'
        if modele_key not in performances:
            performances[modele_key] = {}
        performances[modele_key][f'Code_type_local {code_type_local}'] = lasso_rmse

        modele_key = f'Ridge alpha={alpha}'
        if modele_key not in performances:
            performances[modele_key] = {}
        performances[modele_key][f'Code_type_local {code_type_local}'] = ridge_rmse

# Créez un DataFrame à partir du dictionnaire de performances
df_performances = pd.DataFrame(performances)

df_performances.head()

,ElasticNet alpha=10.0 l1_ratio=0.1,ElasticNet alpha=10.0 l1_ratio=0.5,ElasticNet alpha=10.0 l1_ratio=0.9,ElasticNet alpha=0.1 l1_ratio=0.1,ElasticNet alpha=0.1 l1_ratio=0.5,ElasticNet alpha=0.1 l1_ratio=0.9,ElasticNet alpha=1.0 l1_ratio=0.1,ElasticNet alpha=1.0 l1_ratio=0.5,ElasticNet alpha=1.0 l1_ratio=0.9,Lasso alpha=10.0,Ridge alpha=10.0,Lasso alpha=0.1,Ridge alpha=0.1,Lasso alpha=1.0,Ridge alpha=1.0
Code_type_local 1,64900.857408,64700.470802,64190.029580,63928.631984,63919.470825,63912.641835,64164.962349,64054.309673,63931.206926,63911.465534,63911.457797,63911.434525,63911.434468,63911.437180,63911.436584
Code_type_local 2,64663.608803,64617.609197,64509.993920,64443.603118,64442.806580,64443.802067,64504.070032,64476.116163,64443.981108,64443.927710,64444.394543,64444.440606,64444.445313,64444.393625,64444.440659
Code_type_local 3,90296.755813,90280.017500,90233.000983,90206.522768,90209.091593,90213.740969,90229.859224,90215.395339,90206.108160,90215.009175,90215.218595,90215.343308,90215.345417,90215.312754,90215.333839
Code_type_local 4,105527.672503,105523.868607,105477.368068,105338.655812,105321.246175,105300.906949,105470.975437,105434.224261,105342.769611,105297.164081,105296.491630,105295.269402,105295.262771,105295.441358,105295.374963


In [71]:
# Résultat : meilleurs modèles par Code_type_local avec les paramètres optimaux et les modèles eux-mêmes
print(meilleurs_modeles)


{1: {'Modèle': Ridge(alpha=0.1), 'RMSE': 63911.43446763371, 'Alpha': 0.1}, 2: {'Modèle': ElasticNet(alpha=0.1), 'RMSE': 64442.80657979914, 'Alpha': 0.1, 'L1_ratio': 0.5}, 3: {'Modèle': ElasticNet(l1_ratio=0.9), 'RMSE': 90206.10815985709, 'Alpha': 1.0, 'L1_ratio': 0.9}, 4: {'Modèle': Ridge(alpha=0.1), 'RMSE': 105295.26277144227, 'Alpha': 0.1}}


In [72]:
print(X.columns)

Index(['Surface_reelle_bati', 'Nombre_pieces_principales', 'Surface_terrain',
       'month', 'latitude', 'longitude', 'niveau_vie_commune',
       'Prix_moyen_m2'],
      dtype='object')


In [74]:
# Utilisez le modèle avec le RMSE minimum pour faire des prédictions
modele_code_type_1 = meilleurs_modeles[1]['Modèle']
modele_code_type_2 = meilleurs_modeles[2]['Modèle']
modele_code_type_3 = meilleurs_modeles[3]['Modèle']
modele_code_type_4 = meilleurs_modeles[4]['Modèle']

In [75]:
# Export Model

import pickle as pkl

with open('../model/modele_regression_code_1.pkl','wb') as fichier_pickler:
    pkl.dump(modele_code_type_1,fichier_pickler)

with open('../model/modele_regression_code_2.pkl','wb') as fichier_pickler:
    pkl.dump(modele_code_type_2,fichier_pickler)

with open('../model/modele_regression_code_3.pkl','wb') as fichier_pickler:
    pkl.dump(modele_code_type_3,fichier_pickler)

with open('../model/modele_regression_code_4.pkl','wb') as fichier_pickler:
    pkl.dump(modele_code_type_4,fichier_pickler)
